In [234]:
from collections import defaultdict
from functools import cache
import pandas as pd
import pyodbc
import requests
import heapq
import numpy as np
from scipy.spatial import distance
import torch
from torch.utils.data import Dataset, DataLoader
from math import floor
from torch import nn
from numpy import float32
from sklearn.preprocessing import StandardScaler

import time

Fetch rows from the database. Run once and only when needed. 

Load in the data in a dataframe.

In [235]:
df: pd.DataFrame = pd.read_parquet('bikeshare_data.parquet.gzip')  
print(df.shape[0])

8240287


Next, we fetch station_information, which will be used in later functions

In [236]:
BIKE_BASE_ENDPOINT = "https://tor.publicbikesystem.net/ube/gbfs/v1/en/" 
ENDPOINTS = {
        "station_status": BIKE_BASE_ENDPOINT + "station_status",
        "station_information": BIKE_BASE_ENDPOINT + "station_information",
}


def _fetch_station_status(endpoints: dict[str, str]) -> list[dict]:
    """Return a sorted list (by station_id) containing station status for all stations."""
    response = requests.get(endpoints["station_status"])
    response_data: dict = response.json()
    return response_data["data"]["stations"]

def _fetch_station_information(endpoints: dict[str, str]) -> list[dict]:
    """Return a sorted list (by station_id) containing station information for all stations."""
    response = requests.get(endpoints["station_information"])
    response_data: dict = response.json()
    return response_data["data"]["stations"]

station_status = _fetch_station_status(ENDPOINTS)
station_information = _fetch_station_information(ENDPOINTS)


Load the working station IDS, discarding EOL stations and other define ID related functions

In [237]:
def get_all_IDs() -> list[str]:
    """Return a sorted list with all the stationIDs."""
    ids = []
    status_i = 0
    information_i = 0
    while status_i < len(station_status) and information_i < len(station_information):
        status_station_id = int(station_status[status_i]["station_id"])
        information_station_id = int(station_information[information_i]["station_id"])
        if status_station_id < information_station_id:
            status_i += 1
        elif information_station_id < status_station_id:
            information_i += 1
        else:
            ids.append(station_status[status_i]["station_id"])
            status_i += 1
            information_i += 1
    
    return ids

original_station_ids = get_all_IDs()
NUM_STATIONS_ORIGINAL = len(original_station_ids)


# to be used for testing
def id_to_original_ID(id: int) -> int:
    """Convert the id (internal) to the original ID in the data."""
    return original_station_ids[id]

@cache
def station_encoding(station_id: int) -> list[int]:
    """Return a one hot encoding of the original station ID, station_id, based on its index in original_station_ids."""
    one_hot_encoding = [0] * NUM_STATIONS_ORIGINAL
    index = original_station_ids.index(station_id)
    one_hot_encoding[index] = 1
    return tuple(one_hot_encoding)  


Weather related code

In [238]:
# list of all weather status, taking from the open weather map website
WEATHER_STATUSES = [
        200, 201, 202, 210, 211, 212, 221, 230, 231, 232,
        300, 301, 302, 310, 311, 312, 313, 314, 321,
        500, 501, 502, 503, 504, 511, 520, 521, 522, 531,
        600, 601, 602, 611, 612, 613, 615, 616, 620, 621, 622,
        701, 711, 721, 731, 741, 751, 761, 762, 771, 781,
        800,
        801, 802, 803, 804
]
NUM_STATUSES =  len(WEATHER_STATUSES)

@cache
def weather_encoding(weather_status: int) -> tuple[int]:
    """Return a one hot encoding of weather_status based on its index in WEATHER_STATUSES."""
    one_hot_encoding = [0] * NUM_STATUSES
    index = WEATHER_STATUSES.index(weather_status)
    one_hot_encoding[index] = 1
    return tuple(one_hot_encoding)



In [239]:
# Certain stations are turned of for a day or some period of time, we ignore those stations in our analysis

group_sizes = df.groupby("StationID").size()
size_to_groups: dict = group_sizes.groupby(group_sizes).apply(lambda x: list(x.index)).to_dict()

station_ids = size_to_groups[max(size_to_groups.keys())]
NUM_STATIONS = len(station_ids)


In [240]:
station_ids_set = set(station_ids)

def nearest_k_stations(K: int) -> dict[int, list[int]]:
    """Compute the K nearest stations to all stations, not including itself.
    
    The returned dict is a mapping from (transformed) stationID to a list containing the 
    (transformed) station IDs of the nearest K stations.
    """
    ids = []
    coordinates = []
    status_i = 0
    information_i = 0
    while status_i < len(station_status) and information_i < len(station_information):
        status_station_id = int(station_status[status_i]["station_id"])
        information_station_id = int(station_information[information_i]["station_id"])
        if status_station_id < information_station_id:
            status_i += 1
        elif information_station_id < status_station_id:
            information_i += 1
        else:
            if station_status[status_i]["station_id"] in station_ids_set:
                # the tranformed stationID is the stationID used internally
                # it is simply the index of the stationID in the original_station_ids list
                transformed_station_id = len(ids)
                ids.append(transformed_station_id)
                coordinates.append((station_information[information_i]["lat"], station_information[information_i]["lon"]))
            status_i += 1
            information_i += 1
    
    k_nearest = {}
    for i, id in enumerate(ids):
        coordinate = coordinates[i]
        distances = [(distance.euclidean(x, coordinate), j) for j, x in enumerate(coordinates)]
        closest = heapq.nsmallest(K + 1, distances)
        k_nearest[id] = [ids[j] for _, j in closest[1:]]
    return k_nearest


def nearest_k_stations_inclusive(K: int) -> dict[int, list[int]]:
    """Compute the K nearest stations to all stations, including itself.
    
    The returned dict is a mapping from (transformed) stationID to a list containing the 
    (transformed) station IDs of the nearest K stations.
    """
    ids = []
    coordinates = []
    status_i = 0
    information_i = 0
    while status_i < len(station_status) and information_i < len(station_information):
        status_station_id = int(station_status[status_i]["station_id"])
        information_station_id = int(station_information[information_i]["station_id"])
        if status_station_id < information_station_id:
            status_i += 1
        elif information_station_id < status_station_id:
            information_i += 1
        else:
            if station_status[status_i]["station_id"] in station_ids_set:
                # the tranformed stationID is the stationID used internally
                # it is simply the index of the stationID in the original_station_ids list
                transformed_station_id = len(ids)
                ids.append(transformed_station_id)
                coordinates.append((station_information[information_i]["lat"], station_information[information_i]["lon"]))
            status_i += 1
            information_i += 1
    
    k_nearest = {}
    for i, id in enumerate(ids):
        coordinate = coordinates[i]
        distances = [(distance.euclidean(x, coordinate), j) for j, x in enumerate(coordinates)]
        closest = heapq.nsmallest(K + 1, distances)
        k_nearest[id] = [ids[j] for _, j in closest]
    return k_nearest

print(df.columns)

Index(['StationID', 'Time', 'NumBikes', 'NumDocks', 'Latitude', 'Longitude',
       'Temperature', 'WeatherStatus'],
      dtype='object')


Preprocess the data in the data frame. Further preprocessing will be done later.

In [241]:
# Remove station ids that are now removed from the endpoints, but were present when data collection started (1 station was problematic)
df = df[df["StationID"].isin(station_ids_set)]

df = df.sort_values(by=["Time", "StationID"])

# df['StationID'] = df['StationID'].apply(station_encoding)
# df['WeatherStatus'] = df['WeatherStatus'].apply(weather_encoding)

# turn categorical variables into a one-hot encoding
# df = pd.get_dummies(df, columns=["WeatherStatus", "StationID"], drop_first=True)
df = pd.get_dummies(df, columns=["WeatherStatus",], drop_first=True)
# df = df.drop('StationID', axis=1)

# Extract year, month, day of the year, hours, and minutes
df['year'] = df['Time'].dt.year
df['month'] = df['Time'].dt.month
df['dayoftheyear'] = df['Time'].dt.dayofyear
df['hours'] = df['Time'].dt.hour + (df['Time'].dt.minute / 60)
df['weekday'] = df['Time'].dt.weekday 
df = df.drop('Time', axis=1)

# Copy the original NumBikes column for the target
df['NumBikes_original'] = df['NumBikes'].copy()

# maintain this list whenever a new numeric column is added
numeric_cols = ['NumBikes', 'NumDocks', 'Latitude', 'Longitude', 'Temperature', 'year', 'month', 'dayoftheyear', 'hours', 'weekday']
feature_scaler = StandardScaler()
df[numeric_cols] = feature_scaler.fit_transform(df[numeric_cols])



In [242]:
df.columns
# TODO: check why this is different
# df.head(820)
# df.iloc[820]["StationID"]

Index(['StationID', 'NumBikes', 'NumDocks', 'Latitude', 'Longitude',
       'Temperature', 'WeatherStatus_701', 'WeatherStatus_741',
       'WeatherStatus_800', 'WeatherStatus_801', 'WeatherStatus_802',
       'WeatherStatus_803', 'WeatherStatus_804', 'year', 'month',
       'dayoftheyear', 'hours', 'weekday', 'NumBikes_original'],
      dtype='object')

Split data into train test validation

In [243]:
# Constants for splitting
TRAIN_RATIO = 0.70
VALIDATION_RATIO = 0.15
TEST_RATIO = 1 - TRAIN_RATIO - VALIDATION_RATIO

# Ensure the ratios sum to 1
assert TRAIN_RATIO + VALIDATION_RATIO + TEST_RATIO == 1.0

# Calculate the indices for the splits
n = len(df) // NUM_STATIONS
train_end = floor(TRAIN_RATIO * n) * NUM_STATIONS  # First half for training
val_end = floor((TRAIN_RATIO + VALIDATION_RATIO) * n) * NUM_STATIONS  # Next quarter for validation

# Perform the split
df_train = df.iloc[:train_end]
df_val = df.iloc[train_end:val_end]
df_test = df.iloc[val_end:]

assert df_train.shape[0] + df_val.shape[0] + df_test.shape[0] == df.shape[0]
assert df_train.shape[0] % NUM_STATIONS == 0
assert df_val.shape[0] % NUM_STATIONS == 0 
assert df_test.shape[0] % NUM_STATIONS == 0 
# df_train, df_val, and df_test are your resulting splits



Create the torch Dataset to train the model

In [244]:
# TODO:  fix later (broken after normalization), Optimized version still works
class BikeShareDataset(Dataset):  
    def __init__(self, data: pd.DataFrame, lookback: int, lookback_size: int, horizon: int, k: int) -> None:
        """
        Args:
            data (pd.DataFrame): DataFrame containing time series data.
            lookback (int): Number of time steps to look back.
            lookback_size (int): Length of time to jump over each lookback
            horizon (int): Number of time steps to predict.
            k (int):: Number of nearest stations information to consider in predictions.
        """
        self.FLAT_NUM_FEATURES = data.shape[1] * (k + 1)
        self.data = data.to_numpy()
        self.lookback = lookback
        self.lookback_size = lookback_size
        self.horizon = horizon
        self.k = k
        self.k_nearest_dict = nearest_k_stations(k)
        self.k_nearest_differences = self._compute_k_nearest_differences()

    def _compute_k_nearest_differences(self) -> dict[int, list[int]]:
        """Return the differences between the indices of neighbours in the pandas dataframe."""
        k_nearest_differences = defaultdict(list)
        for station in self.k_nearest_dict:
            for near_station in self.k_nearest_dict[station]:
                diff = near_station - station
                k_nearest_differences[station].append(diff) 
        return k_nearest_differences
    
    def __len__(self) -> int:
        FIRST_MINUTES_SKIPPED = self.lookback * self.lookback_size
        padded_length = len(self.data) - (NUM_STATIONS * (self.horizon + FIRST_MINUTES_SKIPPED))
        padded_length = max(padded_length, 0)
        return padded_length
            
    def __getitem__(self, idx: int):
        FIRST_MINUTES_SKIPPED = self.lookback * self.lookback_size
        start_i = NUM_STATIONS * FIRST_MINUTES_SKIPPED
        idx = idx + start_i     
        current_station = idx % NUM_STATIONS
        x = np.ndarray((self.lookback, self.k + 1, self.data.shape[1]))
        for steps_behind in range(self.lookback):
            for k in range(self.k + 1):
                if k == 0:
                    x[steps_behind][k] = self.data[idx - (steps_behind * self.lookback_size * NUM_STATIONS)]
                else: 
                    x[steps_behind][k] = self.data[idx - (steps_behind * self.lookback_size * NUM_STATIONS) + self.k_nearest_differences[current_station][k - 1]]
        x = x.reshape(self.lookback, self.FLAT_NUM_FEATURES)

        y = np.ndarray((self.horizon))
        for minutes_ahead in range(1, self.horizon + 1):
            y[minutes_ahead - 1] = self.data[idx + (NUM_STATIONS * minutes_ahead)][0]  # index 0 is NumBikes
        return x, y
        

class BikeShareDatasetOptimized(Dataset):
    def __init__(self, data: pd.DataFrame, lookback: int, lookback_size: int, horizon: int, k: int) -> None:
        """
        Args:
            data (pd.DataFrame): DataFrame containing time series data.
            lookback (int): Number of time steps to look back.
            lookback_size (int): Length of time to jump over each lookback
            horizon (int): Number of time steps to predict.
            k (int):: Number of nearest stations information to consider in predictions.
        """
        self.target_data = data['NumBikes_original'].to_numpy(dtype=float)
        self.data = data.drop(columns=['NumBikes_original']).to_numpy(dtype=float)
        self.FLAT_NUM_FEATURES = self.data.shape[1] * (k + 1)
        self.lookback = lookback
        self.lookback_size = lookback_size
        self.horizon = horizon
        self.k = k
        self.k_nearest_dict = nearest_k_stations_inclusive(k)
        self.k_nearest_differences = self._compute_k_nearest_differences()

    def _compute_k_nearest_differences(self) -> dict[int, list[int]]:
        """Return the differences between the indices of neighbours in the pandas dataframe."""
        k_nearest_differences = defaultdict(np.array)
        for station in self.k_nearest_dict:
            near_stations =  np.array(self.k_nearest_dict[station])
            diff = near_stations - station
            k_nearest_differences[station] = diff
            
        return k_nearest_differences
    
    def __len__(self) -> int:
        FIRST_MINUTES_SKIPPED = self.lookback * self.lookback_size
        padded_length = len(self.data) - (NUM_STATIONS * (self.horizon + FIRST_MINUTES_SKIPPED))
        padded_length = max(padded_length, 0)
        return padded_length
            
    def __getitem__(self, idx: int):
        FIRST_MINUTES_SKIPPED = self.lookback * self.lookback_size
        start_i = NUM_STATIONS * FIRST_MINUTES_SKIPPED
        idx = idx + start_i     
        current_station = idx % NUM_STATIONS
        
        indices = np.arange(idx, idx - (self.lookback - 1) * self.lookback_size * NUM_STATIONS - 1, -(self.lookback_size * NUM_STATIONS))
        neighbour_diffs = self.k_nearest_differences[current_station]
        x = self.data[indices[:, None] + neighbour_diffs[None, :]]
        x = x.reshape(self.lookback, self.FLAT_NUM_FEATURES)
        
        current_occupancy = self.target_data[idx] # used for debugging

        y_indices = idx + np.arange(1, self.horizon + 1) * NUM_STATIONS
        y = self.target_data[y_indices] 
        return x, y, current_occupancy
        


In [245]:
lookback = 6
lookback_size = 5
horizon = 120
k = 5
train_data = BikeShareDatasetOptimized(df_train, lookback, lookback_size, horizon, k)
validation_data = BikeShareDatasetOptimized(df_val, lookback, lookback_size, horizon, k)
test_data = BikeShareDatasetOptimized(df_test, lookback, lookback_size, horizon, k)

batch_size = 124
train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
validation_dataloader = DataLoader(validation_data, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=True)



Timing Experiments on the Datasets

In [246]:

# data = BikeShareDataset(df_train, lookback, lookback_size, horizon, k)
# optimized_data = BikeShareDatasetOptimized(df_train, lookback, lookback_size, horizon, k)

# def compare_get_item(indices: np.ndarray) -> None:
#     total_time = 0
#     total_time_optimized = 0

#     for idx in indices:
#         start = time.perf_counter()  # Use perf_counter for high-resolution timing
#         X, y = data[idx]
#         end = time.perf_counter()
#         total_time += end - start

#         start = time.perf_counter()  # Use perf_counter for high-resolution timing
#         Xo, yo = optimized_data[idx]
#         end = time.perf_counter()
#         total_time_optimized += end - start
        
#         assert np.array_equal(X, Xo)
#         assert np.array_equal(y, yo)

#     print(f"BikeShareDataset's __getitem took {total_time} seconds in total")
#     print(f"BikeShareDatasetOptimized's __getitem took {total_time_optimized} seconds in total")


# random_indices = np.random.randint(low=0, high=len(data), size=10_000)
# compare_get_item(random_indices)


In [247]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")


Using cpu device


In [248]:
# FLAT_NUM_FEATURES = df.shape[1] * (k + 1)
FLAT_NUM_FEATURES = (df.shape[1] - 1) * (k + 1)  # remove NumBikes_original

class BikeShareLSTM(nn.Module):
    def __init__(self, hidden_size: int, horizon: int, lookback: int, num_layers: int):
        super().__init__()
        self.lookback = lookback
        self.flatten = nn.Flatten()
        self.lstm = nn.LSTM(input_size=FLAT_NUM_FEATURES, 
                            hidden_size=hidden_size, 
                            num_layers=num_layers, 
                            batch_first=True
        )
        self.hidden_size = hidden_size
        # self.linear = nn.Linear((hidden_size * self.lookback), horizon)
        self.linear = nn.Linear(hidden_size, horizon)

    def forward(self, x):
        x, _ = self.lstm(x)
        # x is of shape (batch_size, lookbacks, hidden_size)
        x = self.linear(x[:, -1, :])
        # x = x.reshape(batch_size, sequence_length * self.hidden_size)
        # x = self.linear(x)
        return x

class BikeShareNN(nn.Module):
    def __init__(self, hidden_size: int, horizon: int, lookback: int):
        super().__init__()
        self.flatten = nn.Flatten()
        self.lookback = lookback
        self.horizon = horizon
        self.hidden_size = hidden_size
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(FLAT_NUM_FEATURES * lookback, self.hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, self.horizon),
        )
        
    def forward(self, x):
        assert x.shape[1] * x.shape[2] == FLAT_NUM_FEATURES * self.lookback
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


class TrivialNN(nn.Module):
    def __init__(self, horizon: int, lookback: int):
        super().__init__()
        self.flatten = nn.Flatten()
        self.lookback = lookback
        self.horizon = horizon
        
    def forward(self, x):
        batch_size = x.shape[0]
        result = torch.zeros((batch_size, self.horizon))
        for i in range(batch_size):
            current_occupancy = x[i, 0, 0]  # Assuming current occupancy is in the first column of features
            result[i] = torch.zeros((self.horizon)) + current_occupancy
        return result

        

In [249]:
HIDDEN_SIZE = 50
NUM_LAYERS = 3

# model = BikeShareLSTM(HIDDEN_SIZE, horizon, lookback, NUM_LAYERS).to(device)
model = BikeShareNN(HIDDEN_SIZE, horizon, lookback).to(device)
# model = TrivialNN(horizon, lookback).to(device)
print(model)


BikeShareNN(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=648, out_features=50, bias=True)
    (1): ReLU()
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): ReLU()
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): ReLU()
    (6): Linear(in_features=50, out_features=120, bias=True)
  )
)


In [252]:

def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y, current_occupancies) in enumerate(dataloader):
        X = X.to(device)
        y = y.to(device)
        # Compute prediction and loss
        pred = model(X)
        assert pred.shape == y.shape
        # total_loss_by_time = torch.zeros((horizon), dtype=float)
        loss = loss_fn(pred, y)
        # total_loss_by_time += torch.sum(torch.abs(pred - y), dim=0)

        total_delta_from_current_occupancy = torch.zeros((horizon), dtype=float)
        for i, current_occupancy in enumerate(current_occupancies):
            delta = pred[i] - current_occupancy  # Compute delta from current occupancy for each time step
            total_delta_from_current_occupancy += torch.abs(delta)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            # average_loss_by_time = total_loss_by_time / (batch_size)
            average_delta_from_current_occupancy = total_delta_from_current_occupancy / batch_size
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
            # print(f"average loss by time {average_loss_by_time}")
            print(f"average delta from current occupancy {average_delta_from_current_occupancy}")



def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss =  0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()

    test_loss /= num_batches
    print(f"Avg loss: {test_loss:>8f} \n")



In [254]:
learning_rate = 1e-3
epochs = 5

class WeightedL1Loss(nn.Module):
    def __init__(self, weights):
        super(WeightedL1Loss, self).__init__()
        self.weights = weights

    def forward(self, predictions, targets):
        # Ensure the weights are on the same device as predictions and targets
        weights = self.weights.to(predictions.device)
        
        # Calculate the L1 loss (element-wise absolute difference)
        loss = torch.abs(predictions - targets)
        
        # Apply the weights to the loss components
        weighted_loss = loss * weights
        
        # Return the mean of the weighted loss
        return torch.mean(weighted_loss)


# weights = torch.linspace(1, 0.1, steps=horizon)


# loss_fn = nn.L1Loss(reduction="mean")
loss_fn = nn.L1Loss(reduction="mean")
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
# optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(validation_dataloader, model, loss_fn)
    
print("Done!")

Epoch 1
-------------------------------


KeyboardInterrupt: 